# Exploring DeFi Concepts: Token Wrapping, Staking & Arbitrage Signals


This notebook explores some core DeFi mechanisms and simple algorithmic ideas using Python. Concepts include token wrapping, staking reward simulation, and a basic arbitrage signal logic.

## Table of Contents
TBD 

In [ ]:
from web3 import Web3
import solcx

In [ ]:
# simple web3.py example using EthereumTesterProvider
# This script connects to a local Ethereum test network, checks account balances,
# sends a transaction, and retrieves the transaction details.

w3 = Web3(Web3.EthereumTesterProvider())
print(f"connection check: {w3.is_connected()}")
print(f"account number zero: {w3.eth.accounts[0]}")
balance = w3.eth.get_balance(w3.eth.accounts[0])
print(f"Balance: {Web3.from_wei(balance, 'ether')} ETH")
print(f"Balance in Gwei: {Web3.from_wei(balance, 'gwei')} Gwei")
print(f"Balance in Wei: {balance} Wei")

w3.eth.get_block('latest')

tx_hash = w3.eth.send_transaction({
   'from': w3.eth.accounts[0],
   'to': w3.eth.accounts[1],
   'value': w3.to_wei(3, 'ether'),
   'gas': 21000,  
   'gasPrice': w3.to_wei(20, 'gwei')
})

print(f"Transaction Hash: {tx_hash.hex()}")
print(f"Transaction Details: {w3.eth.get_transaction(tx_hash)}")
print(f"New Balance of acc 0: {Web3.from_wei(w3.eth.get_balance(w3.eth.accounts[0]), 'ether')} ETH")

print(f"New Balance of acc 1: {Web3.from_wei(w3.eth.get_balance(w3.eth.accounts[1]), 'ether')} ETH")

## Concept 1: Lock-and-Mint Simulation (Wrapped Tokens)

In [ ]:
solcx.install_solc('0.8.0') 
solcx.set_solc_version('0.8.0')

In [ ]:
# Solidity code for a simple storage contract
# This contract allows storing a favorite number and associating names with numbers.
# It also provides functions to retrieve the stored number and add people with their favorite numbers.  \
    
solidity_code = ''' 
pragma solidity >=0.6.0 <0.9.0;

contract SimpleStorage {

    uint256 favoriteNumber;

    struct People {
        uint256 favoriteNumber;
        string name;
    }

    People[] public people;
    mapping(string => uint256) public nameToFavoriteNumber;

    function store(uint256 _favoriteNumber) public {
        favoriteNumber = _favoriteNumber;
    }
    
    function retrieve() public view returns (uint256){
        return favoriteNumber;
    }

    function addPerson(string memory _name, uint256 _favoriteNumber) public {
        people.push(People(_favoriteNumber, _name));
        nameToFavoriteNumber[_name] = _favoriteNumber;
    }
}
'''

# Compile the Solidity code

compiled_sol = solcx.compile_source(
    solidity_code,
    output_values=["abi", "bin"]
)

# Get contract interface
contract_id, contract_interface = compiled_sol.popitem()

# Access ABI and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

# Print the ABI for verification
print("Contract ABI:")  
from pprint import pprint
pprint(abi)


In [ ]:
# Set default account for transactions
w3.eth.default_account = w3.eth.accounts[0]

# Create contract in Python
SimpleStorage = w3.eth.contract(abi=abi, bytecode=bytecode)

# Deploy the contract
tx_hash = SimpleStorage.constructor().transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# Get the deployed contract instance
contract_instance = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)

print("Contract deployed at:", tx_receipt.contractAddress)

# Call store() to set favoriteNumber = 42
tx_hash = contract_instance.functions.store(42).transact()
w3.eth.wait_for_transaction_receipt(tx_hash)

# Call retrieve() to get it back
retrieved_number = contract_instance.functions.retrieve().call()
print("Retrieved favorite number:", retrieved_number)

# Add a person to the array
tx_hash = contract_instance.functions.addPerson("Alice", 99).transact()
w3.eth.wait_for_transaction_receipt(tx_hash)

# Check the mapping
fav_number = contract_instance.functions.nameToFavoriteNumber("Alice").call()
print("Alice's favorite number:", fav_number)

# Access the first entry in the people array
person = contract_instance.functions.people(0).call()
print("Person stored:", person)